In [ ]:
import re
import pandas as pd
from collections import Counter

import numpy as np
import tqdm

In [ ]:
# articles
!gdown 1BAIEjxgx-Q0hGV9ByLajBd-5Id2NZQE5

Downloading...
From: https://drive.google.com/uc?id=1BAIEjxgx-Q0hGV9ByLajBd-5Id2NZQE5
To: /content/articles.csv
100% 25.2M/25.2M [00:00<00:00, 39.1MB/s]


In [ ]:
articles = pd.read_csv('articles.csv')
articles.head()

,date,title,body,count,url
0,2024-02-23,Зеленский без плана Б и с угрозой паралича эко...,Президент Украины Владимир Зеленский не смог о...,673,/mezhdunarodnaya-panorama/20061391
1,2024-02-23,Как советские летчики впервые долетели напряму...,"24 февраля 1899 года, 125 лет назад, родился в...",179,/infographics/9497
2,2024-02-23,"Провал ""украинизации"" G20 и капризы Запада. За...","Запад стремится наказать всех тех, кто не след...",472,/politika/20062849
3,2024-02-23,Военная операция на Украине. Хроника событий 2...,Госдепартамент США внес в свои санкционные спи...,59,/armiya-i-opk/20054555
4,2024-02-23,Военная операция на Украине. Хроника событий 2...,Минобороны РФ объявило об освобождении населен...,51,/armiya-i-opk/20043925


**отделим тела новостей, оставим только буквы, токенизируем, приведём к нижнему регистру**

In [ ]:
corpus = articles.body.values
print(len(corpus), type(corpus))
print(corpus[0])


TOKEN_PATTERN = re.compile('[а-яА-ЯёЁ]+')


def tokenize(text):
    return re.findall(TOKEN_PATTERN, text.lower())


docs = [tokenize(text) for text in corpus]
print(docs[0])

2396 <class 'numpy.ndarray'>
Президент Украины Владимир Зеленский не смог объяснить, в чем заключается его план Б на тот случай, если Вашингтон не предоставит Киеву новый пакет помощи. У украинских властей нет выбора, проводить или не проводить мобилизацию, так как часть подразделений Вооруженных сил Украины (ВСУ) укомплектованы лишь наполовину. Однако, по словам экспертов, мобилизация уже создала огромный дефицит рабочей силы, а дополнительный призыв сотен тысяч человек приведет к частичному параличу экономики. ТАСС собрал актуальную информацию о событиях на Украине и вокруг нее. Вооруженные силы РФ за прошедшие сутки заняли более выгодные рубежи и позиции и отразили девять атак и контратак ВСУ на авдеевском направлении, три атаки на купянском направлении и три атаки на запорожском направлении, сообщили в Минобороны РФ. Потери противника за сутки на всех направлениях, включая донецкое, южнодонецкое и херсонское, составили до 840 бойцов. Российские средства ПВО за минувшие сутки сбили 

### исходные данные

**посмотрим на самые частотные слова**

In [ ]:
def occurencies_calculate(corpus):
    occurencies = Counter([token for doc in corpus for token in doc])
    return occurencies

occurencies = occurencies_calculate(docs)
print(f'len(occurencies) = {len(occurencies)}')

total_count = sum(occurencies.values())
print(f'total_count = {total_count}')

occurencies.most_common(30)

len(occurencies) = 127638
total_count = 1814663


[('в', 76261),
 ('и', 55003),
 ('на', 33455),
 ('с', 21631),
 ('что', 21263),
 ('не', 20275),
 ('по', 17062),
 ('это', 9664),
 ('для', 9393),
 ('из', 9220),
 ('как', 9000),
 ('о', 8951),
 ('а', 8923),
 ('к', 8668),
 ('за', 7960),
 ('но', 6816),
 ('он', 6189),
 ('то', 5848),
 ('от', 5818),
 ('мы', 5731),
 ('его', 5697),
 ('у', 5457),
 ('все', 5180),
 ('также', 4787),
 ('года', 4590),
 ('россии', 4289),
 ('до', 4153),
 ('рф', 4052),
 ('при', 3930),
 ('будет', 3807)]

**сколько раз пары слов встречались вместе**

In [ ]:
def cooccurencies_calculate(corpus):
    cooccurencies = Counter([(doc[i], doc[i + 1]) for doc in corpus for i in range(len(doc) - 1)])
    return cooccurencies


cooccurencies = cooccurencies_calculate(docs)
cooccurencies.most_common(20)

[(('в', 'году'), 2617),
 (('в', 'том'), 2231),
 (('а', 'также'), 1923),
 (('при', 'этом'), 1557),
 (('и', 'в'), 1546),
 (('том', 'числе'), 1543),
 (('из', 'за'), 1242),
 (('что', 'в'), 1233),
 (('у', 'нас'), 1224),
 (('в', 'этом'), 1194),
 (('о', 'том'), 1120),
 (('том', 'что'), 1094),
 (('на', 'украине'), 1020),
 (('кроме', 'того'), 1002),
 (('не', 'только'), 979),
 (('по', 'его'), 969),
 (('потому', 'что'), 912),
 (('и', 'не'), 902),
 (('то', 'что'), 812),
 (('в', 'россии'), 802)]

**PMI**

In [ ]:
def pmi_calculate(occurencies, cooccurencies, total_count):
    pmis = {}

    for elem in cooccurencies:
        numerator = cooccurencies[elem] * total_count
        denominator = occurencies[elem[0]] * occurencies[elem[1]]
        pmis[elem] = np.log2(numerator/denominator)

    sorted_pmis = sorted(pmis.items(), key=lambda item: -item[1])
    return sorted_pmis


pmi_scores = pmi_calculate(occurencies, cooccurencies, total_count)
pmi_scores[:50]

[(('станкозавод', 'саста'), 20.791270220371178),
 (('ганновере', 'фрайбурге'), 20.791270220371178),
 (('подсказал', 'тамерлану'), 20.791270220371178),
 (('самаркандский', 'ферганский'), 20.791270220371178),
 (('ферганский', 'ташкентский'), 20.791270220371178),
 (('ташкентский', 'кашкадарьинский'), 20.791270220371178),
 (('кашкадарьинский', 'бухарский'), 20.791270220371178),
 (('ароматнейшего', 'туй'), 20.791270220371178),
 (('тандыра', 'глинобитные'), 20.791270220371178),
 (('узбекским', 'пловом'), 20.791270220371178),
 (('нарезанная', 'кубиками'), 20.791270220371178),
 (('дашогузском', 'велаятах'), 20.791270220371178),
 (('сушеный', 'урюк'), 20.791270220371178),
 (('осетриной', 'белугой'), 20.791270220371178),
 (('суханберды', 'кермолла'), 20.791270220371178),
 (('туршу', 'говурма'), 20.791270220371178),
 (('курагой', 'вяленой'), 20.791270220371178),
 (('альбухарой', 'изюмом'), 20.791270220371178),
 (('изюмом', 'сябзи'), 20.791270220371178),
 (('сладком', 'соусе'), 20.791270220371178)

In [ ]:
pmi_scores[-50:]

[(('от', 'в'), -6.3487326886070505),
 (('в', 'от'), -6.3487326886070505),
 (('у', 'и'), -6.369844796498907),
 (('по', 'а'), -6.390542052382298),
 (('о', 'по'), -6.395062080824536),
 (('в', 'этого'), -6.396774129570183),
 (('в', 'она'), -6.416782108072633),
 (('на', 'для'), -6.436034297272082),
 (('по', 'из'), -6.4377799631302555),
 (('на', 'на'), -6.461243305138086),
 (('для', 'по'), -6.464599221673381),
 (('по', 'это'), -6.505633667138634),
 (('в', 'были'), -6.51352383325521),
 (('из', 'в'), -6.597938643544601),
 (('в', 'также'), -6.652293698947667),
 (('по', 'с'), -6.668041669920987),
 (('из', 'не'), -6.686695108937521),
 (('и', 'но'), -6.690662088352103),
 (('том', 'и'), -6.707914881587119),
 (('и', 'этом'), -6.7158406513481),
 (('с', 'из'), -6.780093557052955),
 (('из', 'с'), -6.780093557052955),
 (('в', 'что'), -6.803445149076434),
 (('в', 'во'), -6.8035129975040745),
 (('не', 'не'), -6.823558367612818),
 (('в', 'того'), -6.823992389409759),
 (('на', 'он'), -6.834154610019897),
 (

**t-score**

в [статье](https://blogs.helsinki.fi/slavica-helsingiensia/files/2019/11/sh34-21.pdf) написано, что перед подсчётом t-score стоит убрать стоп-слова

In [ ]:
def t_score_calculate(occurencies, cooccurencies, total_count):
    t_scores = {}

    for elem in cooccurencies:
        numerator = cooccurencies[elem] - (occurencies[elem[0]] * occurencies[elem[1]]) / total_count
        denominator = np.sqrt(cooccurencies[elem])
        t_scores[elem] = numerator/denominator

    sorted_t_scores = sorted(t_scores.items(), key=lambda item: -item[1])

    return sorted_t_scores


t_scores = t_score_calculate(occurencies, cooccurencies, total_count)
t_scores[:50]

[(('в', 'году'), 48.05712357199485),
 (('в', 'том'), 44.164794599881255),
 (('а', 'также'), 43.31525333140421),
 (('при', 'этом'), 39.26849875131381),
 (('том', 'числе'), 39.19907383193491),
 (('у', 'нас'), 34.82231237541915),
 (('из', 'за'), 34.094429180626506),
 (('о', 'том'), 32.958054657207775),
 (('том', 'что'), 31.853834335167374),
 (('кроме', 'того'), 31.597780856271367),
 (('на', 'украине'), 30.75695981614743),
 (('в', 'этом'), 30.33652980815207),
 (('не', 'только'), 30.26842223971559),
 (('потому', 'что'), 29.824530250146765),
 (('по', 'его'), 29.40801282753272),
 (('тасс', 'собрал'), 27.834861731184304),
 (('заявил', 'что'), 27.304760633824394),
 (('в', 'результате'), 27.023335507810536),
 (('по', 'данным'), 26.744653183106582),
 (('по', 'словам'), 26.672390828837443),
 (('то', 'что'), 26.09093030388688),
 (('его', 'словам'), 25.79527799582884),
 (('со', 'стороны'), 25.679549966393214),
 (('во', 'время'), 25.28964170905792),
 (('в', 'ходе'), 24.801130417888714),
 (('в', 'част

In [ ]:
t_scores[-50:]

[(('у', 'и'), -115.54362919770921),
 (('в', 'если'), -116.54359735113351),
 (('не', 'что'), -116.7844037708379),
 (('при', 'и'), -118.11952246780807),
 (('к', 'в'), -118.42389486826663),
 (('в', 'было'), -120.62001098826615),
 (('на', 'для'), -121.03454162007661),
 (('из', 'и'), -122.74272103515997),
 (('в', 'из'), -126.15647331396151),
 (('в', 'украины'), -126.50349458825137),
 (('в', 'а'), -129.74986221936908),
 (('на', 'и'), -132.00782284130761),
 (('в', 'чтобы'), -135.49681951965738),
 (('для', 'в'), -136.7331409363627),
 (('на', 'не'), -138.63301465689636),
 (('от', 'в'), -139.4305336811777),
 (('в', 'от'), -139.4305336811777),
 (('в', 'также'), -140.83665970676952),
 (('по', 'с'), -142.39792774760238),
 (('и', 'но'), -144.6705685986533),
 (('по', 'на'), -155.27691863447924),
 (('не', 'не'), -158.7666916624528),
 (('в', 'по'), -160.13873236064805),
 (('на', 'в'), -163.5778558724102),
 (('при', 'в'), -164.15778962815685),
 (('с', 'в'), -166.50095520997107),
 (('в', 'до'), -173.5293

**log-likelihood**

In [ ]:
for elem in cooccurencies:
    print(elem)
    break

('президент', 'украины')


In [ ]:
O11 = cooccurencies[elem]
O12 = occurencies[elem[0]] - O11
O21 = occurencies[elem[1]] - O11
O22 = total_count + O11 - O12 - O21

N_p1 = O11 + O21
N_p2 = O12 + O22
N_1p = O11 + O12
N_2p = O21 + O22

print(total_count, N_p1 + N_p2, N_1p + N_2p)

1814663 1815073 1815073


In [ ]:
def log_likelihood_calculate(occurencies, cooccurencies, total_count):
    lls = {}
    for elem in cooccurencies:
        O11 = cooccurencies[elem]
        O12 = occurencies[elem[0]] - O11
        O21 = occurencies[elem[1]] - O11
        O22 = total_count + O11 - O12 - O21

        N_p1 = O11 + O21
        N_p2 = O12 + O22
        N_1p = O11 + O12
        N_2p = O21 + O22

        E11 = N_1p * N_p1 / total_count
        E12 = N_1p * N_p2 / total_count
        E21 = N_p1 * N_2p / total_count
        E22 = N_p2 * N_2p / total_count

        lls[elem] = 2 * (O11 * np.log(O11/E11) + O12 * np.log(O12/E12) + O21 * np.log(O21/E21) + O22 * np.log(O22/E22))

    sorted_lls = sorted(lls.items(), key=lambda item: -item[1])
    return sorted_lls

In [ ]:
ll_scores = log_likelihood_calculate(occurencies, cooccurencies, total_count)
ll_scores[:50]

<ipython-input-13-d4d723d36d2d>:19: RuntimeWarning: divide by zero encountered in log
  lls[elem] = 2 * (O11 * np.log(O11/E11) + O12 * np.log(O12/E12) + O21 * np.log(O21/E21) + O22 * np.log(O22/E22))
<ipython-input-13-d4d723d36d2d>:19: RuntimeWarning: invalid value encountered in scalar multiply
  lls[elem] = 2 * (O11 * np.log(O11/E11) + O12 * np.log(O12/E12) + O21 * np.log(O21/E21) + O22 * np.log(O22/E22))


[(('тасс', 'собрал'), 7128.026247322394),
 (('вооруженных', 'сил'), 4736.486191165883),
 (('вооруженные', 'силы'), 3854.604114981517),
 (('минобороны', 'рф'), 3393.5413405311356),
 (('прежде', 'всего'), 2514.88408616509),
 (('по', 'словам'), 1867.5057314992928),
 (('российской', 'федерации'), 1776.547182044474),
 (('владимир', 'зеленский'), 1486.4117419532317),
 (('о', 'событиях'), 1449.4428487769185),
 (('сил', 'украины'), 1409.886221855838),
 (('на', 'украине'), 1299.4946030553015),
 (('за', 'сутки'), 1104.3682855270795),
 (('в', 'течение'), 1041.4363820510498),
 (('купянском', 'направлении'), 971.3259711819152),
 (('вокруг', 'нее'), 931.1711907268942),
 (('прошедшие', 'сутки'), 879.4538699359366),
 (('украины', 'всу'), 869.7646282277957),
 (('информацию', 'о'), 825.9426337635541),
 (('любом', 'случае'), 803.6336710834964),
 (('всех', 'направлениях'), 570.2711942340343),
 (('украины', 'владимир'), 536.484792317407),
 (('президент', 'украины'), 520.2681119121764),
 (('за', 'прошедшие'

In [ ]:
ll_scores[-50:]

[(('лукашенко', 'запад'), 2.4976930923780087),
 (('инфраструктуры', 'отмечается'), 1.916257803977353),
 (('бойню', 'с'), 1.0879805416045283),
 (('отношение', 'запада'), 0.9901401147651341),
 (('выполнено', 'до'), 0.7472750958064007),
 (('взносы', 'которые'), 0.5592866556270204),
 (('на', 'бойню'), 0.2422816479185821),
 (('рф', 'направлять'), -0.08079797687182477),
 (('давались', 'в'), -0.3477236104983614),
 (('для', 'ареста'), -1.9402306750732403),
 (('правительству', 'рф'), -2.467889693437404),
 (('русскими', 'на'), -2.487762200833072),
 (('что', 'поручение'), -2.8148612974015816),
 (('бюджет', 'при'), -3.116274276320534),
 (('меморандума', 'в'), -3.999925518074718),
 (('направлять', 'добровольные'), nan),
 (('добровольные', 'взносы'), nan),
 (('в', 'арбитражный'), -3.3898664111192622),
 (('к', 'автоконцерну'), nan),
 (('автоконцерну', 'газ'), nan),
 (('с', 'фольксваген'), nan),
 (('фольксваген', 'акциенгезельшафт'), nan),
 (('акциенгезельшафт', 'млрд'), nan),
 (('у', 'водителя'), 0.1

### лемматизация

In [ ]:
!pip install pymorphy2==0.8
import pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=225ef6ccfaaf9dc469acaab5bcdcc4d14a9f8be6405c727cb8129b3572f1a0be
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
lemmatizer = pymorphy2.MorphAnalyzer()

In [ ]:
words = [
    'аббревиатур',
    'аббревиатура',
    'аббревиатуре',
    'аббревиатурой',
    'аббревиатуру',
    'аббревиатуры',
    'человек',
    'люди',
    'людьми'
]

In [ ]:
# пример работы лемматайзера
[lemmatizer.parse(word)[0].normal_form for word in words]

['аббревиатура',
 'аббревиатура',
 'аббревиатура',
 'аббревиатура',
 'аббревиатура',
 'аббревиатура',
 'человек',
 'человек',
 'человек']

In [ ]:
lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

lemmatized_docs = [[lemmatize(token) for token in text] for text in tqdm.tqdm(docs)]

100%|██████████| 2396/2396 [01:07<00:00, 35.45it/s]


In [ ]:
print(lemmatized_docs[0])

['президент', 'украина', 'владимир', 'зеленский', 'не', 'смочь', 'объяснить', 'в', 'чем', 'заключаться', 'он', 'план', 'б', 'на', 'тот', 'случай', 'если', 'вашингтон', 'не', 'предоставить', 'киев', 'новый', 'пакет', 'помощь', 'у', 'украинский', 'власть', 'нет', 'выбор', 'проводить', 'или', 'не', 'проводить', 'мобилизация', 'так', 'как', 'часть', 'подразделение', 'вооружённый', 'сила', 'украина', 'всу', 'укомплектовать', 'лишь', 'наполовину', 'однако', 'по', 'слово', 'эксперт', 'мобилизация', 'уже', 'создать', 'огромный', 'дефицит', 'рабочий', 'сила', 'а', 'дополнительный', 'призыв', 'сотня', 'тысяча', 'человек', 'привести', 'к', 'частичный', 'паралич', 'экономика', 'тасс', 'собрать', 'актуальный', 'информация', 'о', 'событие', 'на', 'украина', 'и', 'вокруг', 'нея', 'вооружённый', 'сила', 'рф', 'за', 'прошедшее', 'сутки', 'занять', 'более', 'выгодный', 'рубеж', 'и', 'позиция', 'и', 'отразить', 'девять', 'атака', 'и', 'контратака', 'всу', 'на', 'авдеевский', 'направление', 'три', 'атака'

**посмотрим на самые частотные слова**

In [ ]:
occurencies_lem = occurencies_calculate(lemmatized_docs)
print(f'len(occurencies_lem) = {len(occurencies_lem)}')

total_count_lem = sum(occurencies_lem.values())
print(f'total_count_lem = {total_count_lem}')

occurencies_lem.most_common(30)

len(occurencies_lem) = 50609
total_count_lem = 1814663


[('в', 78919),
 ('и', 55003),
 ('на', 33455),
 ('с', 24008),
 ('что', 21798),
 ('не', 20275),
 ('по', 17062),
 ('быть', 16992),
 ('это', 16260),
 ('он', 14037),
 ('год', 13001),
 ('который', 11933),
 ('о', 10981),
 ('для', 9393),
 ('из', 9233),
 ('тот', 9222),
 ('как', 9000),
 ('они', 8966),
 ('а', 8923),
 ('к', 8807),
 ('весь', 8722),
 ('мы', 8671),
 ('за', 7960),
 ('страна', 7091),
 ('но', 6816),
 ('один', 6628),
 ('россия', 6552),
 ('украина', 6268),
 ('этот', 6151),
 ('свой', 5858)]

**сколько раз пары слов встречались вместе**

In [ ]:
cooccurencies_lem = cooccurencies_calculate(lemmatized_docs)
cooccurencies_lem.most_common(20)

[(('в', 'год'), 3060),
 (('в', 'тот'), 2547),
 (('один', 'из'), 1967),
 (('а', 'также'), 1923),
 (('тот', 'что'), 1729),
 (('и', 'в'), 1618),
 (('при', 'это'), 1559),
 (('тот', 'число'), 1543),
 (('что', 'в'), 1300),
 (('в', 'это'), 1284),
 (('заявить', 'что'), 1245),
 (('из', 'за'), 1242),
 (('не', 'быть'), 1242),
 (('у', 'мы'), 1224),
 (('на', 'украина'), 1219),
 (('о', 'тот'), 1146),
 (('год', 'в'), 1030),
 (('по', 'он'), 1023),
 (('кроме', 'тот'), 1004),
 (('не', 'только'), 979)]

**PMI**

In [ ]:
pmi_scores_lem = pmi_calculate(occurencies_lem, cooccurencies_lem, total_count_lem)
pmi_scores_lem[:50]

[(('станкозавод', 'саста'), 20.791270220371178),
 (('ганновер', 'фрайбурге'), 20.791270220371178),
 (('ароматнейшего', 'туй'), 20.791270220371178),
 (('тандыра', 'глинобитный'), 20.791270220371178),
 (('дашогузском', 'велаятах'), 20.791270220371178),
 (('суханберды', 'кермолла'), 20.791270220371178),
 (('туршу', 'говурма'), 20.791270220371178),
 (('курага', 'вяленый'), 20.791270220371178),
 (('сбрызгивать', 'топлёный'), 20.791270220371178),
 (('севиндж', 'абдуллаев'), 20.791270220371178),
 (('разномастный', 'свора'), 20.791270220371178),
 (('свора', 'хрюкать'), 20.791270220371178),
 (('хрюкать', 'подсвинок'), 20.791270220371178),
 (('подобру', 'поздорову'), 20.791270220371178),
 (('шольцев', 'боррелей'), 20.791270220371178),
 (('разрознить', 'растыркано'), 20.791270220371178),
 (('многовариантность', 'многофакторность'), 20.791270220371178),
 (('экоативисткой', 'евгения'), 20.791270220371178),
 (('евгения', 'чириков'), 20.791270220371178),
 (('фабы', 'умпк'), 20.791270220371178),
 (('р

In [ ]:
pmi_scores_lem[-50:]

[(('по', 'мы'), -6.349211597139232),
 (('к', 'быть'), -6.365732780983727),
 (('у', 'и'), -6.369844796498907),
 (('по', 'к'), -6.371663878364306),
 (('по', 'а'), -6.390542052382298),
 (('в', 'от'), -6.398655771724),
 (('на', 'для'), -6.436034297272082),
 (('по', 'из'), -6.439812699122704),
 (('в', 'с'), -6.443079937526642),
 (('для', 'быть'), -6.458668124292802),
 (('на', 'на'), -6.461243305138086),
 (('для', 'по'), -6.464599221673381),
 (('в', 'говорить'), -6.475405264394839),
 (('не', 'это'), -6.50518371058321),
 (('на', 'мочь'), -6.669116555077787),
 (('о', 'быть'), -6.684019651333586),
 (('из', 'не'), -6.688727844929969),
 (('о', 'по'), -6.689950748714165),
 (('и', 'но'), -6.690662088352103),
 (('в', 'также'), -6.701720925369276),
 (('свой', 'на'), -6.754856481291623),
 (('по', 'с'), -6.818456531411843),
 (('не', 'не'), -6.823558367612818),
 (('этот', 'на'), -6.8252692640158665),
 (('при', 'и'), -6.896266065272234),
 (('в', 'если'), -6.9264793731257015),
 (('с', 'из'), -6.9325411545

**t-score**

In [ ]:
t_scores_lem = t_score_calculate(occurencies_lem, cooccurencies_lem, total_count_lem)
t_scores_lem[:50]

[(('в', 'год'), 45.096072673449804),
 (('один', 'из'), 43.59049987583477),
 (('а', 'также'), 43.31525333140421),
 (('в', 'тот'), 42.520939216697016),
 (('тот', 'число'), 38.97054168004069),
 (('тот', 'что'), 38.91715949705137),
 (('при', 'это'), 38.592319458980555),
 (('у', 'мы'), 34.24040222969141),
 (('заявить', 'что'), 34.22511945465454),
 (('из', 'за'), 34.0928111019382),
 (('о', 'тот'), 32.20416060987896),
 (('на', 'украина'), 31.604453408195607),
 (('кроме', 'тот'), 31.492535380782815),
 (('не', 'только'), 30.26842223971559),
 (('не', 'быть'), 29.855000470727184),
 (('потому', 'что'), 29.81509968965902),
 (('отметить', 'что'), 29.678088607423334),
 (('какой', 'то'), 29.590895583976796),
 (('вооружённый', 'сила'), 29.51496111816715),
 (('по', 'он'), 27.857979624509802),
 (('тасс', 'собрать'), 27.826339893735998),
 (('не', 'мочь'), 27.783457619326725),
 (('владимир', 'путин'), 27.75498336954579),
 (('должный', 'быть'), 27.363964308871658),
 (('по', 'данные'), 26.744653183106582),
 

In [ ]:
t_scores_lem[-50:]

[(('с', 'из'), -121.15263329885494),
 (('из', 'с'), -121.15263329885494),
 (('из', 'и'), -122.91893841663118),
 (('с', 'и'), -122.98176994805463),
 (('свой', 'в'), -125.38109004261398),
 (('в', 'сообщить'), -126.59853813077139),
 (('не', 'это'), -127.0465271999287),
 (('в', 'из'), -130.84654653049446),
 (('этот', 'в'), -131.75231902562624),
 (('на', 'и'), -132.00782284130761),
 (('в', 'а'), -134.37074436196698),
 (('на', 'не'), -138.63301465689636),
 (('в', 'чтобы'), -140.25427806705707),
 (('для', 'в'), -141.59741819349844),
 (('который', 'что'), -142.3409586242735),
 (('с', 'о'), -144.27868149623373),
 (('в', 'от'), -144.40083023100675),
 (('и', 'но'), -144.6705685986533),
 (('в', 'также'), -145.7946699760996),
 (('в', 'стать'), -151.7355371217686),
 (('по', 'на'), -155.27691863447924),
 (('по', 'с'), -158.20124058074978),
 (('не', 'не'), -158.7666916624528),
 (('в', 'по'), -165.87213087373513),
 (('на', 'в'), -168.19180537649893),
 (('при', 'в'), -169.91419729172856),
 (('с', 'в'), 

**log-likelihood**

In [ ]:
ll_scores_lem = log_likelihood_calculate(occurencies_lem, cooccurencies_lem, total_count_lem)
ll_scores_lem[:50]

<ipython-input-13-d4d723d36d2d>:19: RuntimeWarning: divide by zero encountered in log
  lls[elem] = 2 * (O11 * np.log(O11/E11) + O12 * np.log(O12/E12) + O21 * np.log(O21/E21) + O22 * np.log(O22/E22))
<ipython-input-13-d4d723d36d2d>:19: RuntimeWarning: invalid value encountered in scalar multiply
  lls[elem] = 2 * (O11 * np.log(O11/E11) + O12 * np.log(O12/E12) + O21 * np.log(O21/E21) + O22 * np.log(O22/E22))


[(('вооружённый', 'сила'), 7336.004905786216),
 (('тот', 'число'), 7280.0458044626685),
 (('а', 'также'), 6767.55933411822),
 (('тасс', 'собрать'), 6397.427097278254),
 (('кроме', 'тот'), 5615.197056570506),
 (('точка', 'зрение'), 4035.384152533951),
 (('минобороны', 'рф'), 3393.5413405311356),
 (('при', 'это'), 3377.7762709638146),
 (('соединить', 'штат'), 3119.5546585473544),
 (('санкт', 'петербург'), 3022.181128363848),
 (('официальный', 'представитель'), 3011.7619949275777),
 (('по', 'данные'), 2660.6445829088684),
 (('отметить', 'что'), 2125.3990724410633),
 (('летательный', 'аппарат'), 2067.239596786134),
 (('обратить', 'внимание'), 2002.8214015043923),
 (('беспилотный', 'летательный'), 1886.983217808932),
 (('привести', 'к'), 1733.8912143549205),
 (('правоохранительный', 'орган'), 1642.250120453958),
 (('вс', 'рф'), 1603.9361575399903),
 (('средство', 'пво'), 1528.3765538782195),
 (('мария', 'захаров'), 1444.7249401243203),
 (('тот', 'кто'), 1430.292607652095),
 (('дональд', 'тр

In [ ]:
ll_scores_lem[-50:]

[(('эмоция', 'это'), -3.937002343477766),
 (('обещать', 'россия'), -3.952486234048101),
 (('поднять', 'что'), -3.9552911289059307),
 (('что', 'уехать'), -3.9596587961538536),
 (('чтобы', 'про'), -3.962912381451392),
 (('то', 'тяжёлый'), -3.9638448965693227),
 (('забота', 'на'), -3.9665265305844075),
 (('задача', 'регион'), -3.969722714879342),
 (('вы', 'да'), -3.976423581153925),
 (('с', 'личность'), -3.9789956324273743),
 (('следующий', 'слово'), -3.9792369144405595),
 (('партия', 'раз'), -3.979461371273315),
 (('это', 'дональд'), -3.980150316462298),
 (('украина', 'посол'), -3.984501742160231),
 (('они', 'старший'), -3.9851573837071483),
 (('связь', 'действие'), -3.985539840139461),
 (('когда', 'ряд'), -3.9864900641747227),
 (('посол', 'этот'), -3.988999051906488),
 (('и', 'подобрать'), -3.990960269209933),
 (('десятилетие', 'мы'), -3.9924118722736055),
 (('сложно', 'весь'), -3.997059428682308),
 (('назвать', 'помощь'), -3.9979573874321774),
 (('для', 'девять'), -3.9979747413622766),

### удаление стоп-слов

In [ ]:
# используем готовые стоп-слова
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopword_set = set(nltk.corpus.stopwords.words('russian'))
stopword_set

{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [ ]:
cleared_docs = [[token for token in text if token not in stopword_set] for text in tqdm.tqdm(docs)]
print(cleared_docs[0])

100%|██████████| 2396/2396 [00:00<00:00, 11328.37it/s]

['президент', 'украины', 'владимир', 'зеленский', 'смог', 'объяснить', 'заключается', 'план', 'б', 'случай', 'вашингтон', 'предоставит', 'киеву', 'новый', 'пакет', 'помощи', 'украинских', 'властей', 'выбора', 'проводить', 'проводить', 'мобилизацию', 'часть', 'подразделений', 'вооруженных', 'сил', 'украины', 'всу', 'укомплектованы', 'лишь', 'наполовину', 'однако', 'словам', 'экспертов', 'мобилизация', 'создала', 'огромный', 'дефицит', 'рабочей', 'силы', 'дополнительный', 'призыв', 'сотен', 'тысяч', 'человек', 'приведет', 'частичному', 'параличу', 'экономики', 'тасс', 'собрал', 'актуальную', 'информацию', 'событиях', 'украине', 'вокруг', 'вооруженные', 'силы', 'рф', 'прошедшие', 'сутки', 'заняли', 'выгодные', 'рубежи', 'позиции', 'отразили', 'девять', 'атак', 'контратак', 'всу', 'авдеевском', 'направлении', 'атаки', 'купянском', 'направлении', 'атаки', 'запорожском', 'направлении', 'сообщили', 'минобороны', 'рф', 'потери', 'противника', 'сутки', 'направлениях', 'включая', 'донецкое', 'юж

In [ ]:
occurencies_clear = occurencies_calculate(cleared_docs)
print(f'len(occurencies_clear) = {len(occurencies_clear)}')

total_count_clear = sum(occurencies_clear.values())
print(f'total_count_clear = {total_count_clear}')

occurencies_clear.most_common(30)

len(occurencies_clear) = 127487
total_count_clear = 1289542


[('это', 9664),
 ('также', 4787),
 ('года', 4590),
 ('россии', 4289),
 ('рф', 4052),
 ('году', 3773),
 ('которые', 3640),
 ('сша', 3436),
 ('украины', 3034),
 ('страны', 2643),
 ('время', 2545),
 ('тасс', 2486),
 ('который', 2299),
 ('заявил', 2144),
 ('украине', 2045),
 ('лет', 1985),
 ('очень', 1981),
 ('тыс', 1946),
 ('президент', 1903),
 ('стран', 1826),
 ('числе', 1694),
 ('президента', 1693),
 ('будут', 1671),
 ('всу', 1647),
 ('человек', 1629),
 ('словам', 1535),
 ('однако', 1455),
 ('глава', 1445),
 ('области', 1418),
 ('против', 1413)]

**сколько раз пары слов встречались вместе**

In [ ]:
cooccurencies_clear = cooccurencies_calculate(cleared_docs)
cooccurencies_clear.most_common(20)

[(('тасс', 'собрал'), 777),
 (('минобороны', 'рф'), 590),
 (('владимир', 'путин'), 528),
 (('вооруженных', 'сил'), 424),
 (('официальный', 'представитель'), 360),
 (('президента', 'рф'), 356),
 (('вооруженные', 'силы'), 354),
 (('основные', 'события'), 353),
 (('онлайн', 'трансляции'), 322),
 (('прим', 'тасс'), 320),
 (('трансляции', 'тасс'), 319),
 (('пресс', 'секретарь'), 316),
 (('премьер', 'министр'), 315),
 (('тыс', 'человек'), 314),
 (('мид', 'рф'), 307),
 (('точки', 'зрения'), 300),
 (('собрал', 'основное'), 300),
 (('военной', 'операции'), 294),
 (('таким', 'образом'), 276),
 (('иностранных', 'дел'), 274)]

**PMI**

In [ ]:
pmi_scores_clear = pmi_calculate(occurencies_clear, cooccurencies_clear, total_count_clear)
pmi_scores_clear[:50]

[(('станкозавод', 'саста'), 20.298427331365573),
 (('ганновере', 'фрайбурге'), 20.298427331365573),
 (('подсказал', 'тамерлану'), 20.298427331365573),
 (('самаркандский', 'ферганский'), 20.298427331365573),
 (('ферганский', 'ташкентский'), 20.298427331365573),
 (('ташкентский', 'кашкадарьинский'), 20.298427331365573),
 (('кашкадарьинский', 'бухарский'), 20.298427331365573),
 (('бухарский', 'хорезмский'), 20.298427331365573),
 (('ароматнейшего', 'туй'), 20.298427331365573),
 (('тандыра', 'глинобитные'), 20.298427331365573),
 (('узбекским', 'пловом'), 20.298427331365573),
 (('нарезанная', 'кубиками'), 20.298427331365573),
 (('дашогузском', 'велаятах'), 20.298427331365573),
 (('сладкая', 'кислая'), 20.298427331365573),
 (('сушеный', 'урюк'), 20.298427331365573),
 (('осетриной', 'белугой'), 20.298427331365573),
 (('перемешиваю', 'впитает'), 20.298427331365573),
 (('переворачиваю', 'раскладываю'), 20.298427331365573),
 (('раскладываю', 'тарелкам'), 20.298427331365573),
 (('суханберды', 'кер

In [ ]:
pmi_scores_clear[-50:]

[(('также', 'президента'), -2.651845017249834),
 (('это', 'стране'), -2.6915214670486027),
 (('тыс', 'россии'), -2.694294264051093),
 (('года', 'которые'), -2.695573842021654),
 (('которые', 'года'), -2.695573842021654),
 (('года', 'стран'), -2.700322156919818),
 (('лет', 'россии'), -2.7229215613003017),
 (('году', 'который'), -2.7498598041627154),
 (('заявил', 'рф'), -2.7520803178933364),
 (('около', 'это'), -2.7537585991765416),
 (('украине', 'россии'), -2.765883397099093),
 (('это', 'территории'), -2.772867422124246),
 (('российские', 'это'), -2.7791811960564483),
 (('это', 'государства'), -2.8072561476691664),
 (('которые', 'время'), -2.844745344683542),
 (('года', 'рф'), -2.850269565736734),
 (('рф', 'года'), -2.850269565736734),
 (('также', 'лет'), -2.8814020513845118),
 (('это', 'лет'), -2.8949006999898246),
 (('заявил', 'года'), -2.931940297393368),
 (('который', 'россии'), -2.9347890199881617),
 (('это', 'людей'), -2.941385602352313),
 (('путин', 'это'), -2.9526019468245637),


**t-score**

In [ ]:
t_scores_clear = t_score_calculate(occurencies_clear, cooccurencies_clear, total_count_clear)
t_scores_clear[:50]

[(('тасс', 'собрал'), 27.818630951612402),
 (('минобороны', 'рф'), 24.145676568469206),
 (('владимир', 'путин'), 22.94380728147902),
 (('вооруженных', 'сил'), 20.576663223473783),
 (('официальный', 'представитель'), 18.963411270115827),
 (('вооруженные', 'силы'), 18.80078341477192),
 (('основные', 'события'), 18.77424214528209),
 (('президента', 'рф'), 18.586016285743863),
 (('онлайн', 'трансляции'), 17.937235066690533),
 (('прим', 'тасс'), 17.848992889278577),
 (('трансляции', 'тасс'), 17.824628082786234),
 (('пресс', 'секретарь'), 17.758055035988484),
 (('премьер', 'министр'), 17.71135394234342),
 (('тыс', 'человек'), 17.58131731295657),
 (('мид', 'рф'), 17.35660671667449),
 (('точки', 'зрения'), 17.314246301458162),
 (('собрал', 'основное'), 17.30870736681681),
 (('военной', 'операции'), 17.12472428981061),
 (('таким', 'образом'), 16.60153945099857),
 (('иностранных', 'дел'), 16.542323648611024),
 (('специальной', 'военной'), 16.509314936134494),
 (('самом', 'деле'), 16.333450329972

In [ ]:
t_scores_clear[-50:]

[(('рф', 'это'), -5.839137820684804),
 (('российские', 'это'), -5.864626355713889),
 (('это', 'государства'), -5.999520760083812),
 (('которые', 'году'), -6.116527230030185),
 (('которые', 'время'), -6.183790834265189),
 (('также', 'лет'), -6.368658795138119),
 (('это', 'году'), -6.4251219942170685),
 (('также', 'украины'), -6.549736250903533),
 (('заявил', 'года'), -6.631360591589882),
 (('который', 'россии'), -6.646444241443861),
 (('это', 'людей'), -6.681486915509537),
 (('это', 'словам'), -6.719985787278742),
 (('путин', 'это'), -6.741439984118393),
 (('которые', 'это'), -6.81603073607879),
 (('министр', 'это'), -6.868840254912209),
 (('тасс', 'россии'), -7.268403820891448),
 (('также', 'это'), -7.4999190445531525),
 (('это', 'заявил'), -7.5444798328489435),
 (('года', 'которые'), -7.747223023137389),
 (('которые', 'года'), -7.747223023137389),
 (('тасс', 'года'), -7.84867650685282),
 (('это', 'поэтому'), -8.022936825632666),
 (('сша', 'сша'), -8.155262876277003),
 (('который', 'эт

**log-likelihood**

In [ ]:
ll_scores_clear = log_likelihood_calculate(occurencies_clear, cooccurencies_clear, total_count_clear)
ll_scores_clear[:50]

<ipython-input-13-d4d723d36d2d>:19: RuntimeWarning: divide by zero encountered in log
  lls[elem] = 2 * (O11 * np.log(O11/E11) + O12 * np.log(O12/E12) + O21 * np.log(O21/E21) + O22 * np.log(O22/E22))
<ipython-input-13-d4d723d36d2d>:19: RuntimeWarning: invalid value encountered in scalar multiply
  lls[elem] = 2 * (O11 * np.log(O11/E11) + O12 * np.log(O12/E12) + O21 * np.log(O21/E21) + O22 * np.log(O22/E22))


[(('тасс', 'собрал'), 6597.050824995482),
 (('вооруженных', 'сил'), 4446.771286726669),
 (('вооруженные', 'силы'), 3612.728324113038),
 (('иностранных', 'дел'), 2794.565490262891),
 (('ключевые', 'заявления'), 1520.8506080723246),
 (('собрал', 'ключевые'), 1381.6523997866525),
 (('владимир', 'зеленский'), 1362.1380045250535),
 (('сил', 'украины'), 1226.007969071601),
 (('сергей', 'лавров'), 948.4231597592561),
 (('инфографике', 'тасс'), 914.1120362648784),
 (('глобального', 'юга'), 841.7765816082087),
 (('министр', 'иностранных'), 761.1640372749456),
 (('украины', 'всу'), 694.5270712987713),
 (('событиях', 'украине'), 676.0739068906321),
 (('рф', 'сергей'), 499.15420857797807),
 (('целый', 'ряд'), 496.72503717789596),
 (('украине', 'вокруг'), 490.9593919094859),
 (('украины', 'владимир'), 433.82954720054823),
 (('данном', 'случае'), 399.99357472301176),
 (('президент', 'украины'), 382.35929357889336),
 (('большому', 'счету'), 345.9731298345923),
 (('новый', 'пакет'), 299.6814818772623)

In [ ]:
ll_scores_clear[-50:]

[(('регионов', 'контексте'), 1.6086236868837656),
 (('артемовска', 'находится'), 1.30748561817239),
 (('конкретных', 'боевых'), 1.2442339135841243),
 (('премьера', 'индии'), 1.2282548480026367),
 (('специальные', 'группы'), 1.109707031081904),
 (('боевых', 'планов'), 0.9294860816470285),
 (('которое', 'посетил'), 0.6292511121751314),
 (('отношение', 'запада'), 0.33228706768105365),
 (('действий', 'донецком'), -1.6480941868410228),
 (('заявил', 'отношение'), -2.3444574712617765),
 (('запада', 'минским'), nan),
 (('минским', 'соглашениям'), nan),
 (('украине', 'приблизило'), nan),
 (('приблизило', 'начало'), nan),
 (('задач', 'регионов'), 0.4836386830782482),
 (('также', 'фонде'), 0.39022439082397575),
 (('это', 'распространенная'), 0.13600279049673691),
 (('хотя', 'конкретных'), 0.07572895601426399),
 (('безопасности', 'цру'), 0.06764312616389434),
 (('будут', 'согласовать'), -0.13619292937700345),
 (('отказывать', 'это'), -0.37126004092386466),
 (('агентства', 'национальной'), -0.58384

### удаление стоп-слов + лемматизация

In [ ]:
cleared_lemmatized_docs = [[token for token in text if token not in stopword_set] for text in tqdm.tqdm(lemmatized_docs)]
print(cleared_lemmatized_docs[0])

100%|██████████| 2396/2396 [00:00<00:00, 5306.68it/s]

['президент', 'украина', 'владимир', 'зеленский', 'смочь', 'объяснить', 'заключаться', 'план', 'б', 'случай', 'вашингтон', 'предоставить', 'киев', 'новый', 'пакет', 'помощь', 'украинский', 'власть', 'выбор', 'проводить', 'проводить', 'мобилизация', 'часть', 'подразделение', 'вооружённый', 'сила', 'украина', 'всу', 'укомплектовать', 'лишь', 'наполовину', 'однако', 'слово', 'эксперт', 'мобилизация', 'создать', 'огромный', 'дефицит', 'рабочий', 'сила', 'дополнительный', 'призыв', 'сотня', 'тысяча', 'человек', 'привести', 'частичный', 'паралич', 'экономика', 'тасс', 'собрать', 'актуальный', 'информация', 'событие', 'украина', 'вокруг', 'нея', 'вооружённый', 'сила', 'рф', 'прошедшее', 'сутки', 'занять', 'выгодный', 'рубеж', 'позиция', 'отразить', 'девять', 'атака', 'контратака', 'всу', 'авдеевский', 'направление', 'атака', 'купянск', 'направление', 'атака', 'запорожский', 'направление', 'сообщить', 'минобороны', 'рф', 'потеря', 'противник', 'сутки', 'весь', 'направление', 'включая', 'донецк

In [ ]:
occurencies_clear_lem = occurencies_calculate(cleared_lemmatized_docs)
print(f'len(occurencies_clear_lem) = {len(occurencies_clear_lem)}')

total_count_clear_lem = sum(occurencies_clear_lem.values())
print(f'total_count_clear_lem = {total_count_clear_lem}')

occurencies_clear_lem.most_common(30)

len(occurencies_clear_lem) = 50502
total_count_clear_lem = 1285071


[('это', 16260),
 ('год', 13001),
 ('который', 11933),
 ('весь', 8722),
 ('страна', 7091),
 ('россия', 6552),
 ('украина', 6268),
 ('свой', 5858),
 ('мочь', 5520),
 ('также', 4787),
 ('человек', 4783),
 ('российский', 4367),
 ('президент', 4238),
 ('рф', 4052),
 ('время', 3528),
 ('стать', 3512),
 ('военный', 3456),
 ('сша', 3436),
 ('наш', 3414),
 ('ещё', 3216),
 ('заявить', 3112),
 ('большой', 3005),
 ('сообщить', 2934),
 ('украинский', 2635),
 ('самый', 2625),
 ('глава', 2597),
 ('вопрос', 2574),
 ('работа', 2493),
 ('тасс', 2487),
 ('новый', 2485)]

**сколько раз пары слов встречались вместе**

In [ ]:
cooccurencies_clear_lem = cooccurencies_calculate(cleared_lemmatized_docs)
cooccurencies_clear_lem.most_common(20)

[(('вооружённый', 'сила'), 874),
 (('тасс', 'собрать'), 777),
 (('владимир', 'путин'), 773),
 (('президент', 'рф'), 694),
 (('это', 'год'), 615),
 (('минобороны', 'рф'), 590),
 (('премьер', 'министр'), 542),
 (('прошлое', 'год'), 539),
 (('глава', 'государство'), 440),
 (('сила', 'украина'), 429),
 (('сектор', 'газа'), 429),
 (('президент', 'украина'), 417),
 (('весь', 'это'), 415),
 (('официальный', 'представитель'), 403),
 (('собрать', 'основный'), 402),
 (('военный', 'операция'), 394),
 (('наш', 'страна'), 375),
 (('основной', 'событие'), 353),
 (('тысяча', 'человек'), 352),
 (('свой', 'очередь'), 339)]

**PMI**

In [ ]:
pmi_scores_clear_lem = pmi_calculate(occurencies_clear_lem, cooccurencies_clear_lem, total_count_clear_lem)
pmi_scores_clear_lem[:50]

[(('станкозавод', 'саста'), 20.293416639652428),
 (('ганновер', 'фрайбурге'), 20.293416639652428),
 (('ароматнейшего', 'туй'), 20.293416639652428),
 (('тандыра', 'глинобитный'), 20.293416639652428),
 (('дашогузском', 'велаятах'), 20.293416639652428),
 (('суханберды', 'кермолла'), 20.293416639652428),
 (('туршу', 'говурма'), 20.293416639652428),
 (('курага', 'вяленый'), 20.293416639652428),
 (('сбрызгивать', 'топлёный'), 20.293416639652428),
 (('севиндж', 'абдуллаев'), 20.293416639652428),
 (('отстреливаться', 'втащить'), 20.293416639652428),
 (('разномастный', 'свора'), 20.293416639652428),
 (('свора', 'хрюкать'), 20.293416639652428),
 (('хрюкать', 'подсвинок'), 20.293416639652428),
 (('подобру', 'поздорову'), 20.293416639652428),
 (('шольцев', 'боррелей'), 20.293416639652428),
 (('урод', 'бугристый'), 20.293416639652428),
 (('разрознить', 'растыркано'), 20.293416639652428),
 (('многовариантность', 'многофакторность'), 20.293416639652428),
 (('транскраниальную', 'стимуляция'), 20.29341

In [ ]:
pmi_scores_clear_lem[-50:]

[(('свой', 'стать'), -4.000852967762299),
 (('который', 'представитель'), -4.001633173911292),
 (('работать', 'который'), -4.010803759802877),
 (('который', 'ситуация'), -4.010803759802877),
 (('самый', 'это'), -4.053724704750811),
 (('наш', 'украина'), -4.057620167847892),
 (('год', 'направление'), -4.069016506380113),
 (('новый', 'весь'), -4.076056771878828),
 (('это', 'находиться'), -4.080406747403317),
 (('это', 'рф'), -4.095078955389946),
 (('стать', 'украина'), -4.098449954355512),
 (('наш', 'год'), -4.110165678451988),
 (('ситуация', 'год'), -4.134469567868729),
 (('военный', 'россия'), -4.139190504152051),
 (('весь', 'глава'), -4.139656934107587),
 (('глава', 'весь'), -4.139656934107587),
 (('министр', 'который'), -4.15914561112844),
 (('это', 'территория'), -4.1743926167857275),
 (('рф', 'свой'), -4.207194297028079),
 (('который', 'мир'), -4.21287114480237),
 (('который', 'дело'), -4.2171992331711055),
 (('мочь', 'российский'), -4.2294627277933605),
 (('который', 'отношение'),

**t-score**

In [ ]:
t_scores_clear_lem = t_score_calculate(occurencies_clear_lem, cooccurencies_clear_lem, total_count_clear_lem)
t_scores_clear_lem[:50]

[(('вооружённый', 'сила'), 29.49496141681173),
 (('тасс', 'собрать'), 27.8064020272685),
 (('владимир', 'путин'), 27.735245647473732),
 (('президент', 'рф'), 25.836628021753178),
 (('минобороны', 'рф'), 24.145174734155535),
 (('премьер', 'министр'), 23.22500809043596),
 (('прошлое', 'год'), 22.84597080296355),
 (('глава', 'государство'), 20.791006432526196),
 (('сектор', 'газа'), 20.692936473317566),
 (('сила', 'украина'), 20.169745281512856),
 (('официальный', 'представитель'), 20.02236959322208),
 (('собрать', 'основный'), 20.012931094960894),
 (('военный', 'операция'), 19.726817259725916),
 (('президент', 'украина'), 19.40831153119403),
 (('основной', 'событие'), 18.750720129877852),
 (('наш', 'страна'), 18.392106240558196),
 (('точка', 'зрение'), 18.37525390128384),
 (('тысяча', 'человек'), 18.314510359530246),
 (('свой', 'очередь'), 18.24854726723859),
 (('это', 'год'), 18.165848029796386),
 (('специальный', 'военный'), 18.017221847463592),
 (('онлайн', 'трансляция'), 17.964656446

In [ ]:
t_scores_clear_lem[-50:]

[(('глава', 'весь'), -16.626289909273495),
 (('который', 'военный'), -16.796251328500464),
 (('министр', 'который'), -16.86601051615047),
 (('это', 'территория'), -17.05582726557521),
 (('глава', 'это'), -17.239583315471442),
 (('самый', 'год'), -17.36441817337656),
 (('рф', 'свой'), -17.471054128526752),
 (('который', 'мир'), -17.543878898520003),
 (('который', 'дело'), -17.59959410802983),
 (('мочь', 'российский'), -17.758372105510123),
 (('который', 'отношение'), -17.803883209565853),
 (('говорить', 'который'), -17.99888644285024),
 (('военный', 'страна'), -18.070149431432192),
 (('большой', 'который'), -18.316918169489742),
 (('владимир', 'это'), -18.561533954155063),
 (('украина', 'рф'), -18.763838729533234),
 (('весь', 'президент'), -18.925035470767817),
 (('весь', 'украина'), -19.271002146963085),
 (('россия', 'рф'), -19.65932855071821),
 (('который', 'решение'), -19.809630751919542),
 (('это', 'всу'), -19.839486689840484),
 (('российский', 'украина'), -20.300267456039393),
 (('

**log-likelihood**

In [ ]:
ll_scores_clear_lem = log_likelihood_calculate(occurencies_clear_lem, cooccurencies_clear_lem, total_count_clear_lem)
ll_scores_clear_lem[:50]

<ipython-input-13-d4d723d36d2d>:19: RuntimeWarning: divide by zero encountered in log
  lls[elem] = 2 * (O11 * np.log(O11/E11) + O12 * np.log(O12/E12) + O21 * np.log(O21/E21) + O22 * np.log(O22/E22))
<ipython-input-13-d4d723d36d2d>:19: RuntimeWarning: invalid value encountered in scalar multiply
  lls[elem] = 2 * (O11 * np.log(O11/E11) + O12 * np.log(O12/E12) + O21 * np.log(O21/E21) + O22 * np.log(O22/E22))


[(('вооружённый', 'сила'), 6732.787774849932),
 (('тасс', 'собрать'), 5861.1875074677755),
 (('премьер', 'министр'), 3901.5099916768204),
 (('онлайн', 'трансляция'), 3868.6683543522313),
 (('минобороны', 'рф'), 2987.088133467141),
 (('летательный', 'аппарат'), 1962.328505040042),
 (('беспилотный', 'летательный'), 1793.1164809176648),
 (('вс', 'рф'), 1430.2164585788419),
 (('средство', 'пво'), 1395.2606504199844),
 (('пол', 'бой'), 1346.3115243663979),
 (('сегодняшний', 'день'), 1231.458584815073),
 (('владимир', 'зеленский'), 1184.6110064602422),
 (('данный', 'момент'), 1150.227054425254),
 (('год', 'назад'), 1143.7097756228295),
 (('джо', 'байден'), 1086.4734646901047),
 (('купянск', 'направление'), 898.8245374356077),
 (('вокруг', 'нея'), 844.3247124698861),
 (('около', 'тысяча'), 754.8854826581507),
 (('сообщить', 'минобороны'), 737.0018974225525),
 (('живой', 'сила'), 704.3428924701316),
 (('минувший', 'сутки'), 702.8238879996152),
 (('мировой', 'война'), 689.0210860011853),
 (('кр

In [ ]:
ll_scores_clear_lem[-50:]

[(('бывать', 'мочь'), -3.9973301630744515),
 (('наш', 'михаил'), -3.997521893818307),
 (('действие', 'хотя'), -3.9990485979242063),
 (('российский', 'влияние'), -3.999136725591599),
 (('область', 'отмечать'), -3.999269252070301),
 (('сегодня', 'жизнь'), -3.9993617911352444),
 (('год', 'проинформировать'), -3.9995294284696774),
 (('выступать', 'киев'), -3.9997594874026126),
 (('март', 'ещё'), -3.9999217985274935),
 (('требовать', 'сша'), -3.999974252954405),
 (('служба', 'администрация'), -4.187577117080992),
 (('история', 'почему'), -4.227884035188373),
 (('нейтральный', 'страна'), -4.576306002366383),
 (('идти', 'летний'), -4.721020011313968),
 (('юмор', 'это'), -4.779404455180068),
 (('россия', 'дружественный'), -5.366322803650966),
 (('набирать', 'весь'), -5.395741917639217),
 (('местный', 'политика'), -5.788145706724367),
 (('число', 'действующий'), -5.791402149570262),
 (('киев', 'гуманитарный'), -6.0826164073223286),
 (('проект', 'составлять'), -6.08830253073363),
 (('нато', 'пре

### results

In [ ]:
result = pd.DataFrame({
    'original PMI': [elem[0] for elem in pmi_scores[:50]],
    'original t-score': [elem[0] for elem in t_scores[:50]],
    'original LL': [elem[0] for elem in ll_scores[:50]],

    'lemma PMI': [elem[0] for elem in pmi_scores_lem[:50]],
    'lemma t-score': [elem[0] for elem in t_scores_lem[:50]],
    'lemma LL': [elem[0] for elem in ll_scores_lem[:50]],

    'stop_words PMI': [elem[0] for elem in pmi_scores_clear[:50]],
    'stop_words t-score': [elem[0] for elem in t_scores_clear[:50]],
    'stop_words LL': [elem[0] for elem in ll_scores_clear[:50]],

    'lemma_stop_words PMI': [elem[0] for elem in pmi_scores_clear_lem[:50]],
    'lemma_stop_words t-score': [elem[0] for elem in t_scores_clear_lem[:50]],
    'lemma_stop_words LL' : [elem[0] for elem in ll_scores_clear_lem[:50]]
    })

In [ ]:
result

,original PMI,original t-score,original LL,lemma PMI,lemma t-score,lemma LL,stop_words PMI,stop_words t-score,stop_words LL,lemma_stop_words PMI,lemma_stop_words t-score,lemma_stop_words LL
0,"(станкозавод, саста)","(в, году)","(тасс, собрал)","(станкозавод, саста)","(в, год)","(вооружённый, сила)","(станкозавод, саста)","(тасс, собрал)","(тасс, собрал)","(станкозавод, саста)","(вооружённый, сила)","(вооружённый, сила)"
1,"(ганновере, фрайбурге)","(в, том)","(вооруженных, сил)","(ганновер, фрайбурге)","(один, из)","(тот, число)","(ганновере, фрайбурге)","(минобороны, рф)","(вооруженных, сил)","(ганновер, фрайбурге)","(тасс, собрать)","(тасс, собрать)"
2,"(подсказал, тамерлану)","(а, также)","(вооруженные, силы)","(ароматнейшего, туй)","(а, также)","(а, также)","(подсказал, тамерлану)","(владимир, путин)","(вооруженные, силы)","(ароматнейшего, туй)","(владимир, путин)","(премьер, министр)"
3,"(самаркандский, ферганский)","(при, этом)","(минобороны, рф)","(тандыра, глинобитный)","(в, тот)","(тасс, собрать)","(самаркандский, ферганский)","(вооруженных, сил)","(иностранных, дел)","(тандыра, глинобитный)","(президент, рф)","(онлайн, трансляция)"
4,"(ферганский, ташкентский)","(том, числе)","(прежде, всего)","(дашогузском, велаятах)","(тот, число)","(кроме, тот)","(ферганский, ташкентский)","(официальный, представитель)","(ключевые, заявления)","(дашогузском, велаятах)","(минобороны, рф)","(минобороны, рф)"
5,"(ташкентский, кашкадарьинский)","(у, нас)","(по, словам)","(суханберды, кермолла)","(тот, что)","(точка, зрение)","(ташкентский, кашкадарьинский)","(вооруженные, силы)","(собрал, ключевые)","(суханберды, кермолла)","(премьер, министр)","(летательный, аппарат)"
6,"(кашкадарьинский, бухарский)","(из, за)","(российской, федерации)","(туршу, говурма)","(при, это)","(минобороны, рф)","(кашкадарьинский, бухарский)","(основные, события)","(владимир, зеленский)","(туршу, говурма)","(прошлое, год)","(беспилотный, летательный)"
7,"(ароматнейшего, туй)","(о, том)","(владимир, зеленский)","(курага, вяленый)","(у, мы)","(при, это)","(бухарский, хорезмский)","(президента, рф)","(сил, украины)","(курага, вяленый)","(глава, государство)","(вс, рф)"
8,"(тандыра, глинобитные)","(том, что)","(о, событиях)","(сбрызгивать, топлёный)","(заявить, что)","(соединить, штат)","(ароматнейшего, туй)","(онлайн, трансляции)","(сергей, лавров)","(сбрызгивать, топлёный)","(сектор, газа)","(средство, пво)"
9,"(узбекским, пловом)","(кроме, того)","(сил, украины)","(севиндж, абдуллаев)","(из, за)","(санкт, петербург)","(тандыра, глинобитные)","(прим, тасс)","(инфографике, тасс)","(севиндж, абдуллаев)","(сила, украина)","(пол, бой)"
